# Sentiment Analysis in NLP

In [121]:
ai_terms = ["Artificial Intelligence", "Machine Learning", "Deep Learning", "Neural Network", "Natural Language Processing", "Supervised Learning", "Unsupervised Learning", "Reinforcement Learning", "Generative Adversarial Networks", "Convolutional Neural Networks", "Recurrent Neural Networks", "Transfer Learning", "Data Mining", "Big Data", "Algorithm","large language models","llms","robotics","Chatbot", "Robotic Process Automation", "Computer Vision", "Image Recognition", "Speech Recognition", "Text Analytics", "Sentiment Analysis", "Autonomous Vehicles", "Internet of Things", "Edge Computing", "Quantum Computing"]


In [29]:
ai_terms = ["Artificial Intelligence", "Machine Learning", "Deep Learning", "Neural Network", "Natural Language Processing", "Supervised Learning", "Unsupervised Learning", "Reinforcement Learning", "Generative Adversarial Networks", "Convolutional Neural Networks", "Recurrent Neural Networks", "Transfer Learning", "Data Mining", "Big Data", "Algorithm","large language models","llms","robotics","Chatbot", "Robotic Process Automation", "Computer Vision", "Image Recognition", "Speech Recognition", "Text Analytics", "Sentiment Analysis", "Autonomous Vehicles", "Internet of Things", "Edge Computing", "Quantum Computing"]


# Sentiment Analysis

#### Running FinancialBert for sentiment analysis - not great results so tried more sentiment analysis models from hugging face

In [36]:
df_analysis = pd.read_parquet('df_analysis.parquet', engine='pyarrow')
#adding this now to do sentiment over time - get sentiment scores

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load pretrained model and tokenizer
model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis", num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

# Define the maximum length for the text chunks. BERT typically works with a max_length of 512.
max_length = 512

# Function to apply sentiment analysis and manage text length
def get_sentiment_and_score(text):
    # Tokenize the text
    inputs = tokenizer(text, truncation=True, padding='longest', max_length=max_length, return_tensors='pt')
    
    # Get the model's output
    output = model(**inputs)

    # Get the sentiment
    sentiment = output.logits.argmax(dim=1).item()

    # Convert logits to probabilities
    probabilities = torch.nn.functional.softmax(output.logits, dim=-1)

    # Get the confidence score
    confidence_score = probabilities.max(dim=-1).values.item()

    # Convert the sentiment label from integer to string
    if sentiment == 0:
        sentiment = 'negative'
    elif sentiment == 1:
        sentiment = 'neutral'
    else:
        sentiment = 'positive'

    return sentiment, confidence_score

# Apply the function to the 'cleaned_text' column
df_filtered_sample['sentiment_finbert_title'], df_filtered_sample['confidence_score'] = zip(*df_filtered_sample['cleaned_text'].map(get_sentiment_and_score))


In [12]:
#value counts for sentiment column
df_filtered_sample['sentiment'].value_counts()

sentiment
neutral     741
positive    259
Name: count, dtype: int64

### Trying sentiment using siebert/sentiment-roberta-large-english - not good results

In [44]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")
print(sentiment_analysis("I love this!"))



[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


In [55]:
%%time
from transformers import pipeline

# Initialize the pipeline
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

# Split your data into batches
batch_size = 100
batches = [df_filtered_sample['cleaned_text'][i:i + batch_size] for i in range(0,df_filtered_sample['cleaned_text'].shape[0],batch_size)]

# Apply the model to each batch and combine the results
sentiments = []
for batch in batches:
    results = sentiment_analysis(list(batch), truncation=True, padding='longest', max_length=512)
    batch_sentiments = [1 if result['label'] == 'positive' else 0 for result in results]
    sentiments.extend(batch_sentiments)

# Assign the results to your dataframe
df_filtered_sample['sentiment'] = sentiments


CPU times: user 2h 42min 58s, sys: 2.37 s, total: 2h 43min
Wall time: 10min 21s


In [57]:
df_filtered_sample['sentiment'].value_counts()

0    1000
Name: sentiment, dtype: int64

#### roberta-base-sentiment - better results

In [67]:
%%time
from transformers import pipeline

# Initialize the pipeline
sentiment_analysis = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# Split your data into batches
batch_size = 100
batches = [df_filtered_sample['cleaned_text'][i:i + batch_size] for i in range(0,df_filtered_sample['cleaned_text'].shape[0],batch_size)]

# Apply the model to each batch and combine the results
sentiments = []
for batch in batches:
    results = sentiment_analysis(list(batch), truncation=True, padding='longest', max_length=512)
    batch_sentiments = [result['label'] for result in results]  # Directly use the model's output
    sentiments.extend(batch_sentiments)

# Assign the results to your dataframe
df_filtered_sample['sentiment_roberta_base'] = sentiments


CPU times: user 51min 41s, sys: 875 ms, total: 51min 42s
Wall time: 3min 15s


In [68]:
df_filtered_sample['sentiment_roberta_base'].value_counts()
#800 neutral for title

LABEL_1    987
LABEL_0      7
LABEL_2      6
Name: sentiment_roberta_base, dtype: int64

In [69]:
%%time
from transformers import pipeline

# Initialize the pipeline
sentiment_analysis = pipeline("sentiment-analysis", model="shashanksrinath/News_Sentiment_Analysis")

# Split your data into batches
batch_size = 100
batches = [df_filtered_sample['clean_title'][i:i + batch_size] for i in range(0,df_filtered_sample['clean_title'].shape[0],batch_size)]

# Apply the model to each batch and combine the results
sentiments = []
for batch in batches:
    results = sentiment_analysis(list(batch), truncation=True, padding='longest', max_length=512)
    batch_sentiments = [result['label'] for result in results]  # Directly use the model's output
    sentiments.extend(batch_sentiments)

# Assign the results to your dataframe
df_filtered_sample['sentiment_srikanth_title'] = sentiments


CPU times: user 5min 29s, sys: 1.04 s, total: 5min 30s
Wall time: 30.6 s


In [70]:
df_filtered_sample['sentiment_srikanth_title'].value_counts()
#1000 neutral for text

Neutral     923
Positive     40
Negative     37
Name: sentiment_srikanth_title, dtype: int64

#### tried another model called news_sentiment_analysis - not good results 

In [71]:
%%time
from transformers import pipeline

# Initialize the pipeline
sentiment_analysis = pipeline("sentiment-analysis", model="shashanksrinath/News_Sentiment_Analysis")

# Split your data into batches
batch_size = 100
batches = [df_filtered_sample['cleaned_text'][i:i + batch_size] for i in range(0,df_filtered_sample['cleaned_text'].shape[0],batch_size)]

# Apply the model to each batch and combine the results
sentiments = []
for batch in batches:
    results = sentiment_analysis(list(batch), truncation=True, padding='longest', max_length=512)
    batch_sentiments = [result['label'] for result in results]  # Directly use the model's output
    sentiments.extend(batch_sentiments)

# Assign the results to your dataframe
df_filtered_sample['sentiment_srikanth'] = sentiments


CPU times: user 51min, sys: 1.08 s, total: 51min 1s
Wall time: 3min 13s


In [72]:
df_filtered_sample['sentiment_srikanth'].value_counts()
#923 neurtral for title

Neutral    1000
Name: sentiment_srikanth, dtype: int64

In [78]:
%%time
from transformers import pipeline

# Initialize the pipeline
sentiment_analysis = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest",tokenizer = "cardiffnlp/twitter-roberta-base-sentiment-latest")

# Split your data into batches
batch_size = 100
batches = [df_filtered_sample['cleaned_text'][i:i + batch_size] for i in range(0,df_filtered_sample['cleaned_text'].shape[0],batch_size)]

# Apply the model to each batch and combine the results
sentiments = []
for batch in batches:
    results = sentiment_analysis(list(batch), truncation=True, padding='longest', max_length=512)
    batch_sentiments = [result['label'] for result in results]  # Directly use the model's output
    sentiments.extend(batch_sentiments)

# Assign the results to your dataframe
df_filtered_sample['roberta_latest'] = sentiments


CPU times: user 50min 39s, sys: 1.76 s, total: 50min 41s
Wall time: 3min 18s


In [46]:
%%time
from transformers import pipeline

# Initialize the pipeline
sentiment_analysis = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest",tokenizer = "cardiffnlp/twitter-roberta-base-sentiment-latest")

# Split your data into batches
batch_size = 100
batches = [df_filtered_sample['clean_title'][i:i + batch_size] for i in range(0,df_filtered_sample['clean_title'].shape[0],batch_size)]

# Apply the model to each batch and combine the results
sentiments = []
for batch in batches:
    results = sentiment_analysis(list(batch), truncation=True, padding='longest', max_length=512)
    batch_sentiments = [result['label'] for result in results]  # Directly use the model's output
    sentiments.extend(batch_sentiments)

# Assign the results to your dataframe
# df_filtered_sample['roberta_latest_title'] = sentiments


CPU times: user 4min 18s, sys: 1.24 s, total: 4min 19s
Wall time: 25.4 s


In [47]:
def get_sentiment(text):
    # Preprocess and encode the text
    encoded_input = tokenizer(text, truncation=True, max_length=512, return_tensors='pt')

    # Get the model's output
    output = model(**encoded_input)
    scores = output.logits.detach().numpy()

    # Convert logits to probabilities
    probabilities = softmax(scores, axis=-1)[0]

    # Create a dictionary mapping labels to their probabilities
    label_probs = {f'LABEL_{i}': prob for i, prob in enumerate(probabilities)}
    
    # Assign the label with the highest probability that exceeds its threshold
    sentiment_label = 'unclassified'  # Default label if no thresholds are exceeded
    for label, prob in label_probs.items():
        if prob > thresholds[label]:  # Check if the probability exceeds the threshold
            sentiment_label = label
            break  # Stop checking other labels

    return sentiment_label


In [145]:
%%time

# Apply the function to the 'cleaned_title' column
df_filtered_sample['roberta_latest_title'] = df_filtered_sample['clean_title'].apply(get_sentiment)


CPU times: user 5min 41s, sys: 200 ms, total: 5min 41s
Wall time: 21.4 s


#### Decided to use cardiffnlp/twitter-roberta-base-sentiment-latest on title because it is able to seperate positive and negative the best - defined thresholds for negative positive and neutral to create the best seperation.

# Roberta Base Sentiment Latest on Title

In [152]:
%%time
df_analysis['roberta_latest_title_sentiment'] = df_analysis['clean_title'].apply(get_sentiment)


CPU times: user 15h 42min 12s, sys: 33.8 s, total: 15h 42min 46s
Wall time: 58min 55s


In [167]:
df_analysis['roberta_latest_title_sentiment'].value_counts()
#0 is negative 
#2 is positive
#1 is neutral

LABEL_2    100996
LABEL_0     46684
LABEL_1     16417
Name: roberta_latest_title_sentiment, dtype: int64

# Bertopics on positive sentiment news 

In [169]:
#removing neutrals
df_analysis = df_analysis[df_analysis['roberta_latest_title_sentiment'] != 'LABEL_1']


In [175]:
df_positive = df_analysis[df_analysis['roberta_latest_title_sentiment'] == 'LABEL_2']


In [108]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to perform aspect-based sentiment analysis
def perform_aspect_sentiment_analysis(text):
    # Tokenize the text into sentences
    sentences = nltk.sent_tokenize(text)
    
    # Analyze sentiment for each sentence
    sentiment_scores = []
    for sentence in sentences:
        sentiment = sia.polarity_scores(sentence)
        sentiment_scores.append(sentiment['compound'])
    
    # Return the average sentiment score for the text
    return sum(sentiment_scores) / len(sentiment_scores)

# Apply aspect-based sentiment analysis to the 'text' column of your DataFrame
chatgpt_df_1['aspect_sentiment'] = chatgpt_df_1['Document'].apply(perform_aspect_sentiment_analysis)


In [334]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to perform targeted sentiment analysis
def perform_targeted_sentiment_analysis(row, target_entity):
    text = row['Document']
    sentiment_scores = analyzer.polarity_scores(text)
    compound_score = sentiment_scores['compound']
    
    # Check if the target entity is mentioned in the text
    if target_entity.lower() in text.lower():
        if compound_score >= 0.05:
            sentiment = 'positive'
        elif compound_score <= -0.05:
            sentiment = 'negative'
        else:
            sentiment = 'neutral'
        return sentiment
    else:
        return 'not mentioned'

# Example target entity
target_entity = "innovation"

# Perform targeted sentiment analysis on the chatgpt_df dataframe
chatgpt_df['Sentiment_innovation'] = chatgpt_df.apply(perform_targeted_sentiment_analysis, target_entity=target_entity, axis=1)

# Interpret the sentiment analysis results
sentiment_counts = chatgpt_df['Sentiment_innovation'].value_counts()
positive_count = sentiment_counts.get('positive', 0)
negative_count = sentiment_counts.get('negative', 0)
neutral_count = sentiment_counts.get('neutral', 0)

# Calculate the percentage of positive, negative, and neutral sentiment
total_count = len(chatgpt_df)
positive_percentage = (positive_count / total_count) * 100
negative_percentage = (negative_count / total_count) * 100
neutral_percentage = (neutral_count / total_count) * 100

# Interpretation of sentiment analysis results
interpretation = f"In the {total_count} analyzed documents, the sentiment towards {target_entity} is as follows:\n"
interpretation += f"Positive: {positive_count} documents ({positive_percentage:.2f}%)\n"
interpretation += f"Negative: {negative_count} documents ({negative_percentage:.2f}%)\n"
interpretation += f"Neutral: {neutral_count} documents ({neutral_percentage:.2f}%)"

# Print the interpretation
print(interpretation)


In the 5882 analyzed documents, the sentiment towards innovation is as follows:
Positive: 1681 documents (28.58%)
Negative: 11 documents (0.19%)
Neutral: 0 documents (0.00%)


In [313]:
df_topics_negative = df_topics_positives_100
#because im copying code from the negative_sentiment_analysis
#will change back the variable name later